In [ ]:
import pandas as pd
import networkx as nx

In [ ]:
#edge list
edges = pd.read_csv(r"C:\Users\rahar\Desktop\Fall 2025\Project 1\Data\imdb_edgelist.csv")

#personnel info
names = pd.read_csv(r"C:\Users\rahar\Desktop\Fall 2025\Project 1\Data\name.basics.tsv", sep="\t", low_memory=False)


In [ ]:
#Function for normalizing category, actors and non actors.
def bucket_A(prof):
    if pd.isna(prof) or prof == "\\N":
        return "unknown"
    profs = set(p.strip().lower() for p in prof.split(","))
    return "actor" if ("actor" in profs or "actress" in profs) else "non_actor"

In [ ]:
#Applying function to names
names["category"] = names["primaryProfession"].apply(bucket_A)

In [ ]:
#Generate graph from edge file
G = nx.from_pandas_edgelist(edges, source="From", target="To", edge_attr="Strength")


In [ ]:
#Assigning category to each notes using dictionary
cat_dict = dict(zip(names["nconst"], names["category"]))
nx.set_node_attributes(G, cat_dict, "category")


In [ ]:
#Calculating degree centrality and eigenvector centrality
deg_centrality = nx.degree_centrality(G)
eig_centrality = nx.eigenvector_centrality(G, max_iter=500)

In [ ]:
#Combining columns into one data frame
df = pd.DataFrame({
    "node": list(deg_centrality.keys()),
    "degree": list(deg_centrality.values()),
    "eigenvector": [eig_centrality.get(n, 0) for n in deg_centrality.keys()],
    "category": [G.nodes[n].get("category") for n in deg_centrality.keys()]
})

In [33]:
df.head()

,node,degree,eigenvector,category
0,nm3662667,0.000090,0.000051,actor
1,nm1568415,0.000181,0.001992,actor
2,nm3421365,0.000090,0.000051,actor
3,nm3661490,0.000090,0.000051,actor
4,nm0219967,0.001324,0.036085,actor


In [ ]:
#Grouping degree centrality and eigenvector centrality by categories
summary = df.groupby("category")[["degree", "eigenvector"]].mean().sort_values("degree", ascending=False)
print(summary.head(10))

             degree  eigenvector
category                        
actor      0.000186     0.000992
non_actor  0.000125     0.000456
unknown    0.000113     0.000108
